In [1]:
%run ../yp_utils.py

# Initial setup

In [2]:
paper_pmid = 20695822
paper_name = 'landstetter_kuchler_2010' 

In [3]:
datasets = pd.read_csv('extras/YeastPhenome_' + str(paper_pmid) + '_datasets_list.txt', sep='\t', header=None, names=['dataset_id', 'name'])

In [4]:
datasets.set_index('dataset_id', inplace=True)

# Load & process the data

In [8]:
original_data = pd.read_excel('raw_data/Supp_Table2.xlsx', sheet_name='Sheet1', skiprows=4)

In [9]:
print('Original data dimensions: %d x %d' % (original_data.shape))

Original data dimensions: 167 x 4


In [10]:
original_data.head()

,Unnamed: 0,Transport,Unnamed: 2,Unnamed: 3
0,1.0,YAL002W,VPS8,late endosome to vacuole transport
1,2.0,YJR102C,VPS25,telomere maintenance
2,3.0,YMR231W,PEP5,"vacuole fusion, nonautophagic"
3,4.0,YNL246W,VPS75,telomere maintenance
4,5.0,YOR036W,PEP12,Golgi to vacuole transport


In [11]:
original_data['orf'] = original_data['Transport'].astype(str)

In [12]:
# Eliminate all white spaces & capitalize
original_data['orf'] = clean_orf(original_data['orf'])

In [13]:
# Translate to ORFs 
original_data['orf'] = translate_sc(original_data['orf'], to='orf')

In [14]:
# Make sure everything translated ok
t = looks_like_orf(original_data['orf'])
print(original_data.loc[~t,])

             Unnamed: 0                                          Transport  \
index_input                                                                  
23                  NaN                                                NaN   
24                  NaN                             vacuolar acidification   
36                  NaN                                                NaN   
37                  NaN                              mitochondrium related   
61                  NaN                                                NaN   
62                  NaN  chromatin / telomere / transcription / transla...   
88                  NaN                                                NaN   
89                  NaN                                              Other   
114                 NaN                                                NaN   
115                 NaN                               protein modification   
126                 NaN                                         

In [15]:
original_data = original_data.loc[t,:]

In [16]:
original_data['data'] = -1

In [17]:
original_data.set_index('orf', inplace=True)

In [18]:
original_data = original_data[['data']].copy()

In [19]:
original_data = original_data.groupby(original_data.index).mean()

In [20]:
original_data.shape

(139, 1)

# Load & process tested strains

In [21]:
tested = pd.read_excel('raw_data/Yeast_deletions_set.xlsx', sheet_name='Stammliste')

In [22]:
tested.head()

,record #,ORF,strain,batch,row,col,comments,slow growth?,batch.1
0,338.0,YAL068C,BY4741,chr1_1,A,2.0,NaN,NaN,chr1_1
1,339.0,YAL067C,BY4741,chr1_1,A,3.0,NaN,NaN,chr1_1
2,340.0,YAL066W,BY4741,chr1_1,A,4.0,NaN,NaN,chr1_1
3,341.0,YAL065C,BY4741,chr1_1,A,5.0,NaN,NaN,chr1_1
4,345.0,YAL062W,BY4741,chr1_1,A,9.0,NaN,NaN,chr1_1


In [24]:
tested['orf'] = tested['ORF'].astype(str)

In [25]:
tested['orf'] = clean_orf(tested['orf'])

In [26]:
tested['orf'] = translate_sc(tested['orf'], to='orf')

In [27]:
# Make sure everything translated ok
t = looks_like_orf(tested['orf'])
print(tested.loc[~t,])

             record #  ORF strain batch  row  col comments slow growth?  \
index_input                                                               
1527              NaN  NaN    NaN   NaN  NaN  NaN      NaN          NaN   
3144              NaN  NaN    NaN   NaN  NaN  NaN      NaN          NaN   

            batch.1  orf  
index_input               
1527            NaN  NAN  
3144            NaN  NAN  


In [28]:
tested = tested.loc[t,:]

In [29]:
tested_orfs = tested['orf'].unique()

In [30]:
missing = [orf for orf in original_data.index.values if orf not in tested_orfs]
missing

[]

In [31]:
original_data = original_data.reindex(index=tested_orfs, fill_value=0)

# Prepare the final dataset

In [32]:
data = original_data.copy()

In [33]:
dataset_ids = [152]
datasets = datasets.reindex(index=dataset_ids)

In [34]:
lst = [datasets.index.values, ['value']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data.columns = idx

In [35]:
data.head()

dataset_id,152
data_type,value
orf,
YAL068C,0
YAL067C,0
YAL066W,0
YAL065C,0
YAL062W,0


## Subset to the genes currently in SGD

In [36]:
genes = pd.read_csv(path_to_genes, sep='\t', index_col='id')
genes = genes.reset_index().set_index('systematic_name')
gene_ids = genes.reindex(index=data.index.values)['id'].values
num_missing = np.sum(np.isnan(gene_ids))
print('ORFs missing from SGD: %d' % num_missing)

ORFs missing from SGD: 22


In [37]:
data['gene_id'] = gene_ids
data = data.loc[data['gene_id'].notnull()]
data['gene_id'] = data['gene_id'].astype(int)
data = data.reset_index().set_index(['gene_id','orf'])

data.head()

,dataset_id,152
,data_type,value
gene_id,orf,
1869,YAL068C,0
61,YAL067C,0
60,YAL066W,0
1727,YAL065C,0
57,YAL062W,0


# Normalize

In [38]:
data_norm = normalize_phenotypic_scores(data, has_tested=True)

In [39]:
# Assign proper column names
lst = [datasets.index.values, ['valuez']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data_norm.columns = idx

In [40]:
data_norm[data.isnull()] = np.nan
data_all = data.join(data_norm)

data_all.head()

dataset_id        152       
data_type       value valuez
gene_id orf                 
1869    YAL068C     0    0.0
61      YAL067C     0    0.0
60      YAL066W     0    0.0
1727    YAL065C     0    0.0
57      YAL062W     0    0.0

# Print out

In [41]:
for f in ['value','valuez']:
    df = data_all.xs(f, level='data_type', axis=1).copy()
    df.columns = datasets['name'].values
    df = df.droplevel('gene_id', axis=0)
    df.to_csv(paper_name + '_' + f + '.txt', sep='\t')

# Save to DB

In [42]:
from IO.save_data_to_db3 import *

In [43]:
save_data_to_db(data_all, paper_pmid)

  0%|          | 0/1 [00:00<?, ?it/s]

Deleting all datasets for PMID 20695822...
Inserting the new data...


100%|██████████| 1/1 [00:08<00:00,  8.42s/it]

Updating the data_modified_on field...
